In [59]:
from langchain_groq import ChatGroq

In [60]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The second person to land on moon was ...")
print(response.content)

The second person to land on the moon was Edwin "Buzz" Aldrin. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission, following Neil Armstrong, who was the first person to set foot on the moon.


In [61]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42414?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than out

In [62]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [63]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer',
 'experience': '5 years of experience in the job offered or in an engineer-related occupation',
 'skills': ['AWS',
  'Databricks',
  'Spark',
  'EMR',
  'CI/CD',
  'Relational Databases',
  'SQL',
  'Python',
  'Scala',
  'Business Intelligence',
  'Data Structures',
  'Cognos',
  'Data Warehousing',
  'Jenkins'],
 'description': 'Develop, code/configure, test programs/systems and solutions problems independently and with minimal supervision in order to meet defined digital product specifications and direction. Develop robust advanced analytics and machine learning solutions that have a direct impact on the business. Advise product owners on discrete technology-related business problems; formulates options, including assessing their relative merits and risks; works with product owners to determine the best solution. Help design and build scalable software solutions, implement and integrate new technologies, manage and evolve existing platforms. Design and 

In [64]:
type(json_res)

dict

In [65]:
import pandas as pd

df = pd.read_csv("/Users/ritikraut/Desktop/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [66]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [71]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-andro

In [72]:
job

{'role': 'Software Engineer',
 'experience': '5 years of experience in the job offered or in an engineer-related occupation',
 'skills': ['AWS',
  'Databricks',
  'Spark',
  'EMR',
  'CI/CD',
  'Relational Databases',
  'SQL',
  'Python',
  'Scala',
  'Business Intelligence',
  'Data Structures',
  'Cognos',
  'Data Warehousing',
  'Jenkins'],
 'description': 'Develop, code/configure, test programs/systems and solutions problems independently and with minimal supervision in order to meet defined digital product specifications and direction. Develop robust advanced analytics and machine learning solutions that have a direct impact on the business. Advise product owners on discrete technology-related business problems; formulates options, including assessing their relative merits and risks; works with product owners to determine the best solution. Help design and build scalable software solutions, implement and integrate new technologies, manage and evolve existing platforms. Design and 

In [73]:
job = json_res
job['skills']

['AWS',
 'Databricks',
 'Spark',
 'EMR',
 'CI/CD',
 'Relational Databases',
 'SQL',
 'Python',
 'Scala',
 'Business Intelligence',
 'Data Structures',
 'Cognos',
 'Data Warehousing',
 'Jenkins']

In [74]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ritik, a Master's student at UAB.
        Over my experience, I have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Ritik
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineer for Scalable Solutions

Dear Hiring Manager,

I am excited to apply for the Software Engineer position at your esteemed organization. With a strong academic background in Computer Science and a Master's degree in progress at the University of Alabama at Birmingham (UAB), I am confident in my ability to develop robust advanced analytics and machine learning solutions that drive business impact.

As a skilled software engineer with a passion for innovation, I have empowered numerous enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. My expertise in AWS, Databricks, Spark, EMR, CI/CD, Relational Databases, SQL, Python, Scala, Business Intelligence, Data Structures, Cognos, Data Warehousing, and Jenkins makes me an ideal fit for this role.

I am impressed by the company's commitment to leveraging technology to drive business growth and would be thrilled to contribute my skil